# core

> Review step state management helpers

In [ ]:
#| default_exp routes.core

In [ ]:
#| export
from typing import List, Dict, Any, NamedTuple, Optional

from cjm_fasthtml_card_stack.core.models import CardStackState
from cjm_fasthtml_card_stack.core.constants import DEFAULT_VISIBLE_COUNT, DEFAULT_CARD_WIDTH

from cjm_workflow_state.state_store import SQLiteWorkflowStateStore

from cjm_transcript_segmentation.models import TextSegment
from cjm_transcript_vad_align.models import VADChunk

from cjm_transcript_review.models import ReviewStepState
from cjm_transcript_review.components.review_card import AssembledSegment

# Type alias for state store
WorkflowStateStore = SQLiteWorkflowStateStore

# Debug flag for state tracing
DEBUG_REVIEW_STATE = False

## Review Context

NamedTuple containing commonly-needed review state values.

In [ ]:
#| export
class ReviewContext(NamedTuple):
    """Common review state values loaded by handlers."""
    segments: List[TextSegment]  # Text segments from segmentation step
    vad_chunks: List[VADChunk]  # VAD chunks from alignment step
    media_path: Optional[str]  # Path to audio file
    focused_index: int  # Currently focused segment index
    visible_count: int  # Number of visible cards in viewport
    is_auto_mode: bool  # Whether card count is in auto-adjust mode
    card_width: int  # Card stack width in rem

## State Getters

In [ ]:
#| export
def _get_review_state(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str  # Session identifier string
) -> ReviewStepState:  # Review step state dictionary
    """Get the review step state from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("review", {})

def _get_segmentation_state(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str  # Session identifier string
) -> Dict[str, Any]:  # Segmentation step state dictionary
    """Get the segmentation step state from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("segmentation", {})

def _get_alignment_state(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str  # Session identifier string
) -> Dict[str, Any]:  # Alignment step state dictionary
    """Get the alignment step state from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("alignment", {})

## Load Context

Loads all review state in a single call, including cross-step data.

In [ ]:
#| export
def _load_review_context(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str  # Session identifier string
) -> ReviewContext:  # Common review state values
    """Load commonly-needed review state values including cross-step data."""
    review_state = _get_review_state(state_store, workflow_id, session_id)
    seg_state = _get_segmentation_state(state_store, workflow_id, session_id)
    align_state = _get_alignment_state(state_store, workflow_id, session_id)
    
    # Get segments from segmentation step
    segment_dicts = seg_state.get("segments", [])
    segments = [TextSegment.from_dict(s) for s in segment_dicts]
    
    # Get VAD chunks from alignment step
    chunk_dicts = align_state.get("vad_chunks", [])
    vad_chunks = [VADChunk.from_dict(c) for c in chunk_dicts]
    
    return ReviewContext(
        segments=segments,
        vad_chunks=vad_chunks,
        media_path=align_state.get("media_path"),
        focused_index=review_state.get("focused_index", 0),
        visible_count=review_state.get("visible_count", DEFAULT_VISIBLE_COUNT),
        is_auto_mode=review_state.get("is_auto_mode", False),
        card_width=review_state.get("card_width", DEFAULT_CARD_WIDTH),
    )

## Assembled Segments

Pairs segments with VAD chunks for display.

In [ ]:
#| export
def _get_assembled_segments(
    ctx:ReviewContext  # Loaded review context
) -> List[AssembledSegment]:  # Paired segments with VAD chunks
    """Pair segments with VAD chunks for review display."""
    return [
        AssembledSegment(segment=seg, vad_chunk=chunk)
        for seg, chunk in zip(ctx.segments, ctx.vad_chunks)
    ]

## State Updates

In [ ]:
#| export
def _update_review_state(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    session_id:str,  # Session identifier string
    focused_index:int=None,  # Updated focused index (None = don't change)
    visible_count:int=None,  # Visible card count (None = don't change)
    is_auto_mode:bool=None,  # Auto-adjust mode flag (None = don't change)
    card_width:int=None,  # Card stack width in rem (None = don't change)
    document_title:str=None,  # Document title (None = don't change)
    is_validated:bool=None,  # Validation flag (None = don't change)
) -> None:
    """Update the review step state in the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    review_state = step_states.get("review", {})
    
    # Update only provided fields
    if focused_index is not None:
        review_state["focused_index"] = focused_index
    if visible_count is not None:
        review_state["visible_count"] = visible_count
    if is_auto_mode is not None:
        review_state["is_auto_mode"] = is_auto_mode
    if card_width is not None:
        review_state["card_width"] = card_width
    if document_title is not None:
        review_state["document_title"] = document_title
    if is_validated is not None:
        review_state["is_validated"] = is_validated
    
    step_states["review"] = review_state
    workflow_state["step_states"] = step_states
    
    if DEBUG_REVIEW_STATE:
        print(f"[REVIEW_STATE] Updated: {review_state}")
    
    state_store.update_state(workflow_id, session_id, workflow_state)

## Card Stack State

In [ ]:
#| export
def _build_card_stack_state(
    ctx:ReviewContext,  # Loaded review context
) -> CardStackState:  # Card stack state for library functions
    """Build a CardStackState from review context for library calls."""
    return CardStackState(
        focused_index=ctx.focused_index,
        visible_count=ctx.visible_count,
        card_width=ctx.card_width,
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()